# Chapter 13: Loading and Preprocessing Data with TensorFlow

Sistem Deep Learning seringkali dilatih pada dataset yang sangat besar yang tidak muat dalam RAM. Bab ini berfokus pada cara efisien untuk memuat dan memproses data dalam jumlah besar menggunakan TensorFlow.

Kita akan membahas:
* **Data API:** Untuk membangun *pipeline* input yang sangat efisien dan dapat diskalakan.
* **TFRecord Format:** Format biner pilihan TensorFlow untuk menyimpan dan membaca data dalam jumlah besar.
* **Preprocessing:** Cara menangani berbagai jenis fitur (numerik, kategorikal, teks) menggunakan lapisan prapemrosesan Keras, baik saat memuat data maupun di dalam model itu sendiri.

## The Data API

Inti dari Data API adalah objek `tf.data.Dataset`, yang merepresentasikan urutan item data. TensorFlow akan menangani semua detail implementasi yang rumit seperti *multithreading*, *queuing*, *batching*, dan *prefetching*.

### Chaining Transformations
Setelah memiliki sebuah *dataset*, kita dapat menerapkan serangkaian transformasi padanya. Setiap metode transformasi mengembalikan *dataset* baru, sehingga kita bisa merantainya.

In [ ]:
import tensorflow as tf

# Membuat dataset dari tensor di memori
dataset = tf.data.Dataset.from_tensor_slices(tf.range(10))

# Merantai transformasi
dataset = dataset.repeat(3).batch(7)

# Iterasi melalui dataset
for item in dataset:
    print(item)

### Shuffling and Prefetching
* **`shuffle(buffer_size)`:** Mengacak data untuk memastikan *instance* pelatihan bersifat independen dan identik terdistribusi (IID).
* **`map(function)`:** Menerapkan fungsi prapemrosesan pada setiap item.
* **`prefetch(n)`:** Membuat *dataset* yang akan selalu berusaha berada `n` *batch* di depan. Ini memungkinkan CPU untuk mempersiapkan data berikutnya saat GPU sedang melatih *batch* saat ini, meningkatkan pemanfaatan perangkat keras secara dramatis.

In [ ]:
# Contoh pipeline yang lengkap
n_steps = 10

# Misalkan kita punya file CSV di path 'Chapter 13/my_data.csv'
# Untuk tujuan konseptual, kita buat dataset dummy
dummy_dataset = tf.data.Dataset.range(100).map(lambda x: (tf.fill([n_steps], x), x))

def build_pipeline(dataset, batch_size=32):
    dataset = dataset.shuffle(1000).repeat()
    dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

train_pipeline = build_pipeline(dummy_dataset)

## The TFRecord Format

**TFRecord** adalah format biner sederhana dan efisien yang disarankan TensorFlow untuk menyimpan data dalam jumlah besar. Ini pada dasarnya adalah daftar catatan biner.

Setiap catatan biasanya berisi *protocol buffer* yang diserialisasi. Protobuf utama yang digunakan adalah `Example`, yang merupakan struktur fleksibel untuk menyimpan fitur-fitur data. Sebuah `Example` berisi pemetaan dari nama fitur ke nilainya (`BytesList`, `FloatList`, atau `Int64List`).

In [ ]:
# Menulis ke file TFRecord
with tf.io.TFRecordWriter("Chapter 13/my_data.tfrecord") as f:
    f.write(b"Ini adalah catatan pertama")
    f.write(b"Dan ini adalah catatan kedua")

# Membaca dari file TFRecord
filepaths = ["Chapter 13/my_data.tfrecord"]
dataset = tf.data.TFRecordDataset(filepaths)
for item in dataset:
    print(item)

## Preprocessing the Input Features

Sebelum data dimasukkan ke jaringan saraf, semua fitur harus diubah menjadi numerik dan seringkali perlu dinormalisasi. Ini bisa dilakukan di awal saat membuat file data, saat memuat dengan Data API, atau sebagai lapisan prapemrosesan langsung di dalam model.

### Encoding Categorical Features
* **One-Hot Encoding:** Cocok untuk fitur dengan sedikit kategori.
* **Embeddings:** Cocok untuk fitur dengan banyak kategori (misalnya, ribuan kata dalam kamus). Embedding adalah vektor padat yang dapat dilatih yang merepresentasikan sebuah kategori. Selama pelatihan, model akan belajar representasi yang berguna untuk setiap kategori.

In [ ]:
from tensorflow import keras

# Konseptual: Menggunakan Embedding Layer
vocab_size = 10000
embed_size = 128

model = keras.models.Sequential([
    # Misalkan input adalah ID kata (integer)
    keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_size, input_shape=[None]),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])

## The TensorFlow Datasets (TFDS) Project

Untuk dataset yang umum digunakan, proyek **TFDS** membuatnya sangat mudah untuk diunduh dan digunakan. Pustaka ini menangani pengunduhan, pemisahan, dan penyajian data sebagai objek `tf.data.Dataset`.

In [ ]:
import tensorflow_datasets as tfds

# Memuat dataset MNIST, sudah siap pakai
dataset, info = tfds.load("mnist", as_supervised=True, with_info=True)
mnist_train = dataset["train"].prefetch(1)

print(info)